In [29]:
class Maze:

    def __init__(self):
        """
        S = Start of maze
        E = End of maze
        _ = Empty space
        # = Wall
        """
        self.maze = [
            ["#", "#", "#", "#", "#", "#", "#", "#"],
            ["#", "S", "#", "#", "#", "#", "_", "#"],
            ["#", "_", "_", "#", "_", "_", "_", "#"],
            ["#", "_", "_", "#", "_", "_", "_", "#"],
            ["#", "_", "_", "#", "_", "_", "_", "#"],
            ["#", "_", "_", "_", "_", "_", "_", "#"],
            ["#", "_", "_", "_", "_", "_", "E", "#"],
            ["#", "#", "#", "#", "#", "#", "#", "#"],
       ]
        
        self.maxMag = 8.0

        self.startLocation = self.FindCharLocation("S")
        self.currentLocation = self.startLocation
        self.endLocation = self.FindCharLocation("E")

    def FindCharLocation(self, targetChar):

        for x in range(len(self.maze)):
            for y in range(len(self.maze[x])):
                if self.maze[x][y] == targetChar:
                    return [x, y]

        return None

    def PrintMaze(self):

        for x in self.maze:
            row = ""
            for y in x:
                row += y
                row += " "

            print(row)
    def RunGenesThroughMaze(self, individual):

        self.currentLocation = self.startLocation

        for aGene in individual.genes:

            possibleMove = None

            if aGene == "1":
                possibleMove = [self.currentLocation[0], self.currentLocation[1] - 1]
            elif aGene == "2":
                possibleMove = [self.currentLocation[0] - 1, self.currentLocation[1]]
            elif aGene == "3":
                possibleMove = [self.currentLocation[0], self.currentLocation[1] + 1]
            elif aGene == "4":
                possibleMove = [self.currentLocation[0] + 1, self.currentLocation[1]]
            else:
                print("Warning! Invalid gene detected: " + aGene)
                
            if self.CheckForVaildMove(possibleMove):
                self.currentLocation = possibleMove
            else:
                individual.CalFitness(self.currentLocation, self.endLocation, self.maxMag)
                break

    def CheckForVaildMove(self, posibleMove):

        if (posibleMove[0] >= 0 and posibleMove[0] < len(self.maze) and
                posibleMove[1] >= 0 and posibleMove[1] < len(self.maze[0])):

            if self.maze[posibleMove[0]][posibleMove[1]] != "#":
                return True

        return False

In [30]:
import random
import math

class Individual:

    MAX_FITNESS = 10
    POSSIBLE_GENES = ["1", "2", "3", "4"]
    GENE_LENGTH = 11

    def __init__(self):
        self.fitness = 0
        self.genes = []

        for x in range(Individual.GENE_LENGTH):
            self.genes.append(random.choice(Individual.POSSIBLE_GENES))
    def CalFitness(self, curLoc, endLoc, maxMag):
        x = curLoc[0] - endLoc[0]
        y = curLoc[1] - endLoc[1]

        x = x ** 2
        y = y ** 2

        mag = math.sqrt(x + y)
        fitMod = 1.0 - (mag / maxMag)
        fitness = Individual.MAX_FITNESS * fitMod
        fitness = int(round(fitness))

        self.fitness = fitness


    def MutateGene(self, index):
          self.genes[index] = random.choice(Individual.POSSIBLE_GENES)

    def CopyGenes(self, otherIndividual):

          for x in range(Individual.GENE_LENGTH):
            self.genes[x] = otherIndividual.genes[x]

            self.fitness = otherIndividual.fitness

    def PrintGenes(self):

        count = 0
        geneOutput = ""

        for item in self.genes:
            geneOutput += item + " "
            count += 1

        return geneOutput

In [31]:
class Population:

    def __init__(self, popSize):

        self.popSize = popSize
        self.individuals = []
        self.fittest = 0

        for x in range(self.popSize):
            temp = Individual()
            self.individuals.append(temp)


    def FindTheFittest(self):

        fittestIndiv = self.individuals[0]

        for item in self.individuals:

            if(item.fitness > fittestIndiv.fitness):
                fittestIndiv = item

        self.fittest = fittestIndiv.fitness

        return fittestIndiv

    def FindTheSecoundFittest(self):

        mostFit = self.individuals[0]
        secoundMostFit = self.individuals[0]

        for item in self.individuals:

            if (item.fitness > mostFit.fitness):
                secoundMostFit = mostFit
                mostFit = item

            elif item.fitness > secoundMostFit.fitness:
                secoundMostFit = item

        return secoundMostFit

    def FindLeastFittest(self):

        leastFitIndiv = self.individuals[0]

        for item in self.individuals:

            if(item.fitness < leastFitIndiv.fitness):
                leastFitIndiv = item

        return leastFitIndiv

    def CalFitForWholePop(self, maze):

        for indiv in self.individuals:
            maze.RunGenesThroughMaze(indiv)

        self.FindTheFittest()

In [32]:
class NaturalSelection:

    def __init__(self, mutationRate):

        self.population = Population(10)
        self.mutationRate = mutationRate
        self.fittest = None
        self.secondFittest = None
        self.genCount = 0

    def MainProcess(self, maze):
        self.population.CalFitForWholePop(maze)
        print("Generation: " + str(self.genCount) + " Fittest: " + str(self.population.fittest))

        while(self.population.fittest < Individual.MAX_FITNESS):
            self.genCount += 1

            self.Selection()
            self.CrossOver()

            if random.uniform(0, 1) <= self.mutationRate:
                self.Mutation()

            self.AddFittestOffspring(maze)
            self.population.CalFitForWholePop(maze)
            print("Generation: " + str(self.genCount) + " Fittest: " + str(self.population.fittest) +
                  " Genes: " + self.population.FindTheFittest().PrintGenes())

        print("Solution found in generation " + str(self.genCount))
        print("Fitness: " + str(self.population.FindTheFittest().fitness))
        print("Genes: " + self.population.FindTheFittest().PrintGenes())

    #Picks the two fittest individuals for the cross over process.
    def Selection(self):

        self.fittest = self.population.FindTheFittest()
        self.secondFittest = self.population.FindTheSecoundFittest()

    def CrossOver(self):

        crossPoint = random.randint(0, Individual.GENE_LENGTH - 1)

        for i in range(crossPoint):
            temp = self.fittest.genes[i]
            self.fittest.genes[i] = self.secondFittest.genes[i]
            self.secondFittest.genes[i] = temp

    def Mutation(self):

        mutaionPoint = random.randint(0, Individual.GENE_LENGTH - 1)

        self.fittest.MutateGene(mutaionPoint)

        mutaionPoint = random.randint(0, Individual.GENE_LENGTH - 1)

        self.secondFittest.MutateGene(mutaionPoint)

    def FindFittestOffspring(self):

        if self.fittest.fitness > self.secondFittest.fitness:
            return self.fittest

        return self.secondFittest

    def AddFittestOffspring(self, maze):

        maze.RunGenesThroughMaze(self.fittest)
        maze.RunGenesThroughMaze(self.secondFittest)

        leastFit = self.population.FindLeastFittest()
        leastFit.CopyGenes(self.FindFittestOffspring())

In [33]:

if __name__ == "__main__":
    NS = NaturalSelection(0.05)
    aMaze = Maze()
    NS.MainProcess(aMaze)


Generation: 0 Fittest: 1
Generation: 1 Fittest: 1 Genes: 2 3 3 1 2 4 3 4 1 4 3 
Generation: 2 Fittest: 1 Genes: 2 3 3 1 2 4 3 4 1 4 3 
Generation: 3 Fittest: 1 Genes: 2 3 3 1 2 4 3 4 1 4 3 
Generation: 4 Fittest: 1 Genes: 2 3 3 1 2 4 3 4 1 4 3 
Generation: 5 Fittest: 1 Genes: 2 3 3 1 2 4 3 4 1 4 3 
Generation: 6 Fittest: 1 Genes: 2 3 3 1 2 4 3 4 1 4 3 
Generation: 7 Fittest: 1 Genes: 2 3 3 1 2 4 3 4 1 4 3 
Generation: 8 Fittest: 1 Genes: 2 3 3 1 2 4 3 4 1 4 3 
Generation: 9 Fittest: 1 Genes: 2 3 3 1 2 4 3 4 1 4 3 
Generation: 10 Fittest: 1 Genes: 2 3 3 1 2 4 3 4 1 4 3 
Generation: 11 Fittest: 1 Genes: 2 3 3 1 2 4 3 4 1 4 3 
Generation: 12 Fittest: 1 Genes: 2 3 3 1 2 4 3 4 1 4 3 
Generation: 13 Fittest: 1 Genes: 2 3 3 1 2 4 3 4 1 4 3 
Generation: 14 Fittest: 1 Genes: 2 3 3 1 2 4 3 4 1 4 3 
Generation: 15 Fittest: 1 Genes: 2 3 3 1 2 4 3 4 2 2 3 
Generation: 16 Fittest: 1 Genes: 2 3 3 1 2 4 3 4 2 2 3 
Generation: 17 Fittest: 1 Genes: 2 3 3 1 2 4 3 4 2 2 3 
Generation: 18 Fittest: 1 Genes:

Generation: 1438 Fittest: 4 Genes: 4 3 4 3 3 3 4 3 2 1 2 
Generation: 1439 Fittest: 4 Genes: 4 3 4 3 3 3 4 3 2 1 2 
Generation: 1440 Fittest: 4 Genes: 4 3 4 3 3 3 4 3 2 1 2 
Generation: 1441 Fittest: 4 Genes: 4 3 4 3 3 3 4 3 2 1 2 
Generation: 1442 Fittest: 4 Genes: 4 3 4 3 1 4 4 2 2 1 2 
Generation: 1443 Fittest: 4 Genes: 4 3 4 3 1 4 4 2 2 1 2 
Generation: 1444 Fittest: 4 Genes: 4 3 4 3 1 4 4 2 2 1 2 
Generation: 1445 Fittest: 4 Genes: 4 3 4 3 1 4 4 2 2 1 2 
Generation: 1446 Fittest: 4 Genes: 4 3 4 3 1 4 4 2 2 1 2 
Generation: 1447 Fittest: 4 Genes: 4 3 4 3 1 4 4 2 2 1 2 
Generation: 1448 Fittest: 4 Genes: 4 3 4 3 1 4 4 2 2 1 2 
Generation: 1449 Fittest: 4 Genes: 4 3 4 3 1 4 4 2 2 1 2 
Generation: 1450 Fittest: 4 Genes: 4 3 4 3 1 4 4 2 2 1 2 
Generation: 1451 Fittest: 4 Genes: 4 3 4 3 1 4 4 2 2 1 2 
Generation: 1452 Fittest: 4 Genes: 4 3 4 3 1 4 4 2 2 1 2 
Generation: 1453 Fittest: 4 Genes: 4 3 4 3 1 4 4 2 2 1 2 
Generation: 1454 Fittest: 4 Genes: 4 3 4 3 1 4 4 2 2 1 2 
Generation: 14

Generation: 2357 Fittest: 9 Genes: 4 4 3 4 1 3 4 1 3 4 2 
Generation: 2358 Fittest: 9 Genes: 4 4 3 4 1 3 4 1 3 4 2 
Generation: 2359 Fittest: 9 Genes: 4 4 3 4 1 3 4 1 3 4 2 
Generation: 2360 Fittest: 9 Genes: 4 4 3 4 1 3 4 1 3 4 2 
Generation: 2361 Fittest: 9 Genes: 4 4 3 4 1 3 4 1 3 4 2 
Generation: 2362 Fittest: 9 Genes: 4 4 3 4 1 3 4 1 3 4 2 
Generation: 2363 Fittest: 9 Genes: 4 4 3 4 1 3 4 1 3 4 2 
Generation: 2364 Fittest: 9 Genes: 4 4 3 4 1 3 4 1 3 4 2 
Generation: 2365 Fittest: 9 Genes: 4 4 3 4 1 3 4 1 3 4 2 
Generation: 2366 Fittest: 9 Genes: 4 4 3 4 1 3 4 1 3 4 2 
Generation: 2367 Fittest: 9 Genes: 4 4 3 4 1 3 4 1 3 4 2 
Generation: 2368 Fittest: 9 Genes: 4 4 3 4 1 3 4 1 3 4 2 
Generation: 2369 Fittest: 9 Genes: 4 4 3 4 1 3 4 1 3 4 2 
Generation: 2370 Fittest: 9 Genes: 4 4 3 4 1 3 4 1 3 4 2 
Generation: 2371 Fittest: 9 Genes: 4 4 3 4 1 3 4 1 3 4 2 
Generation: 2372 Fittest: 9 Genes: 4 4 3 4 1 3 4 1 3 4 2 
Generation: 2373 Fittest: 9 Genes: 4 4 3 4 1 3 4 1 3 4 2 
Generation: 23